# Pauli Transfer Matrix

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm

from itertools import product

import numpy as np; pi = np.pi

import matplotlib.pyplot as plt

import sys; sys.path.append('../../')

from qutip import *
from qutip.qip.circuit import QubitCircuit, Gate
from qutip.qip.operations import *

from qaoa_with_cat_qubits.cvdevice import KPOProcessor

In this notebook we extract the error channels for the Kerr-Nonlinear Resonatr (KNR)

## Setup parameters

In [6]:
## KPO parameters
kpo = KPOProcessor(N=1,num_lvl=20)
alpha = kpo._paras['Coherent state']
num_lvl = kpo._paras['Cut off']

## Cat state
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

## Computational basis
up = (cat_plus + cat_minus)/np.sqrt(2) # Binary 0
down = (cat_plus - cat_minus)/np.sqrt(2) # Binary 1

## Pauli Matrices in computational basis
# Identity
I = up*up.dag() + down*down.dag()
# sigma z
sigma_z = up*up.dag() - down*down.dag()
# sigma x
sigma_x = up*down.dag() + down*up.dag()
# sigma y
sigma_y = 1j*(-up*down.dag() + down*up.dag())
# list of paulis in computational basis
sigma = [I, sigma_x, sigma_y, sigma_z]
# list of paulis in qubit basis
pauli = [qeye(2), sigmax(), sigmay(), sigmaz()]

## RZ-gate

In [52]:
def ideal_ptm(U):
    """ Returns the Pauli Transfer Matrix of a unitary operator U """
    # Get the dimension of U
    a,b = U.dims
    if a != b:
        raise ValueError("The matrix U has to be a square matrix.")
    else: 
        d = a[0] # The dimension of the system
    num_qubits = int(np.log2(d))
    print(num_qubits)
    # Generate all combinations of paulis
    paulis = list(product([identity(2), sigmax(), sigmay(), sigmaz()], repeat=num_qubits))
    M = np.zeros((d**2, d**2), dtype=complex)
    for j in range(d**2):
        print("U",U)
        print("P",paulis[j])
        Lambda = U * paulis[j] * U.dag()
        for i in range(d**2):
            M[i,j] = 1/d * ((paulis[i]*Lambda).tr()).real
    return M

In [7]:
# Angle of rotation
arg = pi/2

# Quantum circuit
qc = QubitCircuit(1)
qc.add_gate("RZ",0,None,arg)

# Matrix representation of the ideal quantum circuit
U_list = qc.propagators()
U = gate_sequence_product(U_list)

d = 2 # dimension
# Pauli transfer matrix
R = np.zeros((d**2,d**2)) 
R_ideal = np.zeros((d**2,d**2)) 

for j in range(d**2):
    result = kpo.run_state(init_state=sigma[j],qc=qc,noisy=True)
    Lambda = result.states[-1]
    Lambda_ideal = U * pauli[j] * U.dag()
    for i in range(d**2):
        R[i,j] = 1/d * ((sigma[i]*Lambda).tr()).real
        R_ideal[i,j] = 1/d * ((pauli[i]*Lambda_ideal).tr()).real

Let $R_U$ correspond to the PTM of the ideal gate $U$, and let $R_\tilde{U}$ correspond to the noisy implementation of $U$.

The PTM of the error channel can thus be extracted by
$$
R_\mathcal{E} = R_U (R_\tilde{U})^{-1}
$$

In [11]:
error_channel = R @ np.linalg.inv(R_ideal)
Qobj(error_channel)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 9.99997559e-01  1.55725985e-06  1.65212774e-06 -5.03503766e-08]
 [ 5.87945699e-06  9.89410242e-01  7.78823474e-03 -4.18128837e-05]
 [ 5.73500444e-06 -7.78823026e-03  9.89410243e-01  4.12750235e-05]
 [-4.99942343e-08  4.12423563e-05 -4.18090131e-05  9.99997555e-01]]

In [14]:
file = 'ptm_error_channel_rz'
np.save(file, error_channel, allow_pickle=True, fix_imports=True)

### Convert PTM to $\chi$-matrix
The relation between the PTM and the $\chi$ matrix can be formalized as
$$
R_{ij} = \frac{1}{d}\sum_{mn}\chi_{mn} \mathrm{Tr}(P_iP_mP_jP_n).
$$
This equation can be vectorized
$$
\vec R = \frac{1}{d} A \vec\chi
$$
where 
$$
A_{(mn,ij)} = \mathrm{Tr}(P_iP_mP_jP_n)
$$
and then solved
$$
d (A^{-1}\vec R)  = \vec\chi
$$

In [34]:
A = np.zeros((4,4,4,4),dtype='complex')
P = [qeye(2),sigmax(),sigmay(),sigmaz()]
for i in range(4):
    for j in range(4):
        for m in range(4):
            for n in range(4):
                A[i,j,m,n] = (P[i]*P[m]*P[j]*P[n]).tr()
A = A.reshape((16,16))
A_inv = np.linalg.inv(A)

In [38]:
chi = d*Qobj((A_inv @ error_channel.reshape((4**2,1))).reshape((4,4)),dims=[[[2],[2]],[[2],[2]]],superrep='chi')
chi.tidyup(1e-3)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = chi
Qobj data =
[[0.9945 0.     0.     0.    ]
 [0.     0.     0.     0.    ]
 [0.     0.     0.     0.    ]
 [0.     0.     0.     0.0055]]

In [23]:
choi = chi_to_choi(chi)
kraus = choi_to_kraus(choi)
kraus = [d*k.full() for k in kraus]

In [20]:
file = 'cv_kraus_rz'
np.save(file, kraus, allow_pickle=True, fix_imports=True)

#### RX

Single angle

In [78]:
# Angle of rotation
arg = pi/2

# Quantum circuit
qc = QubitCircuit(1)
qc.add_gate("RX",0,None,arg)

d = 2 # dimension
# pauli transfer matrix
R = np.zeros((d**2,d**2)) 
R_inv = np.zeros((d**2,d**2)) # inverse/ideal
for j in range(d**2):
    result = kpo.run_state(init_state=sigma[j],qc=qc,noisy=True)
    result_inv = kpo.run_state(init_state=sigma[j],qc=qc,noisy=False)
    
    Lambda = result.states[-1]
    Lambda_inv = result_inv.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * ((sigma[i]*Lambda).tr()).real
        R_inv[i,j] = 1/d * ((sigma[i]*Lambda_inv).tr()).real

In [79]:
error_channel = np.around(R @ np.linalg.inv(R_inv), 3)
Qobj(error_channel)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.     0.     0.     0.   ]
 [ 0.001  0.959  0.     0.   ]
 [ 0.     0.     0.979 -0.004]
 [ 0.     0.    -0.004  0.979]]

In [80]:
A = np.zeros((4,4,4,4),dtype='complex')
P = [qeye(2),sigmax(),sigmay(),sigmaz()]
for i in range(4):
    for j in range(4):
        for m in range(4):
            for n in range(4):
                A[i,j,m,n] = (P[i]*P[m]*P[j]*P[n]).tr()
A = A.reshape((16,16))
A_inv = np.linalg.inv(A)

In [91]:
chi = d*Qobj((A_inv @ error_channel.reshape((4**2,1))).reshape((4,4)),dims=[[[2],[2]],[[2],[2]]],superrep='chi')

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[2.49936159e-04]
 [8.23443556e-03]
 [1.22655644e-02]
 [9.79250064e-01]]

Multiple angles

In [47]:
file = '../../data/ptm/ptm_error_channel_rx.npy'
error_channel = np.load(file)

In [198]:
d = 2
kraus = []
for error in error_channel:
    chi = d*Qobj((A_inv @ error.reshape((4**2,1))).reshape((4,4)),dims=[[[2],[2]],[[2],[2]]],superrep='chi')
    choi = chi_to_choi(chi)
    kraus_list = choi_to_kraus(choi)
    kraus.append([d*k.full() for k in kraus_list])

In [199]:
Qobj(sum(k@np.conjugate(k.T) for k in kraus[1]))

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1.00001999e+00+0.00000000e+00j 1.68922798e-19+1.29054604e-19j]
 [1.68922798e-19-1.29054604e-19j 1.00001999e+00+0.00000000e+00j]]

In [208]:
kraus = np.array(kraus,dtype='object')

In [210]:
file = 'cv_kraus_rx'
np.save(file, kraus, allow_pickle=True, fix_imports=True)

#### $U(\Theta)$-gate (Constant amplitude)

In [43]:
def carb(arg_value):
    # control arbitrary phase gate
    zz = tensor(sigmaz(),sigmaz())
    return (-1j*arg_value/2*zz).expm()

In [44]:
kpo = KPOProcessor(N=2,num_lvl=20) # create two resonators

alpha = kpo._paras['Coherent state']
num_lvl = kpo._paras['Cut off']

## Cat state
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

## Computational basis
up = (cat_plus + cat_minus)/np.sqrt(2) # Binary 0
down = (cat_plus - cat_minus)/np.sqrt(2) # Binary 1

## Pauli Matrices in computational basis
# Identity
I = up*up.dag() + down*down.dag()
# sigma z
sigma_z = up*up.dag() - down*down.dag()
# sigma x
sigma_x = up*down.dag() + down*up.dag()
# sigma y
sigma_y = 1j*(-up*down.dag() + down*up.dag())
sigma = [I, sigma_x, sigma_y, sigma_z]

# Pauli matrices
d = 4
Q = []
for i in range(d):
    Q.extend([tensor(sigma[i], sigma[j]) for j in range(d)])

In [45]:
arg = pi/2

## Create quantum circuit
qc = QubitCircuit(N=1)
qc.user_gates = {"CARB": carb}
qc.add_gate("CARB", targets = [0,1], arg_value = arg)
    
# pauli transfer matrix
R = np.zeros((d**2,d**2))
R_inv = np.zeros((d**2,d**2)) # inverse

for j in range(d**2):
    result = kpo.run_state(init_state=Q[j],qc=qc,noisy=True)
    result_inv = kpo.run_state(init_state=Q[j],qc=qc,noisy=False)
    
    Lambda = result.states[-1]
    Lambda_inv = result_inv.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((Q[i]*Lambda).tr())
        R_inv[i,j] = 1/d * np.real((Q[i]*Lambda_inv).tr())
        
R = Qobj(R,dims=[[2]*2]*2) # Make quantum object
R_inv = Qobj(R_inv,dims=[[2]*2]*2) # Make quantum object


KeyboardInterrupt



In [177]:
error_channel = np.around(R.full() @ np.linalg.inv(R_inv.full()), 3)

### save file

In [165]:
file = 'ptm_error_channel_ising_zz'
np.save(file, error_channel, allow_pickle=True, fix_imports=True)

### load file

In [49]:
file = '../../data/ptm/ptm_error_channel_ising_zz.npy'
error_channel = np.load(file)

In [56]:
num_q = 2

# Pauli matrices
P = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Create all tensor products
sigma = list(map(tensor, map(list, product(P, repeat=num_q))))

In [58]:
A = np.zeros((16,16,16,16),dtype='complex')
for i in tqdm(range(16)):
    for j in range(16):
        for m in range(16):
            for n in range(16):
                A[i,j,m,n] = (sigma[i]*sigma[m]*sigma[j]*sigma[n]).tr()

100%|███████████████████████████████████████████| 16/16 [00:19<00:00,  1.20s/it]


In [61]:
A = A.reshape((16**2,16**2))
A_inv = np.linalg.inv(A)

In [63]:
error_channel.reshape((16**2,1)).shape

(256, 1)

In [ ]:
A_inv.shape

In [ ]:
A_inv @ error_channel

In [66]:
d = 4
chi = d*Qobj((A_inv @ error_channel.reshape((16**2,1))).reshape((16,16)),dims=[[[2,2],[2,2]],[[2,2],[2,2]]],superrep='chi')
chi

Quantum object: dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]], shape = (16, 16), type = super, isherm = True, superrep = chi
Qobj data =
[[9.8925e-01 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 5.2500e-03 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+

In [ ]:
chi

In [70]:
0.989+0.005+0.005+2.5*1e-4

0.99925

In [169]:
choi = chi_to_choi(chi)
kraus = choi_to_kraus(choi)
kraus = [d*k.full() for k in kraus]

In [172]:
Qobj(sum(k@np.conjugate(k.T) for k in kraus))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

In [173]:
kraus

[array([[0.99461048+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
        [0.        +0.j, 0.99461048+0.j, 0.        +0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j, 0.99461048+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.99461048+0.j]]),
 array([[ 1.02469508e-01+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j, -6.56672955e-16+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j, -6.56672955e-16+0.j,
          0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         -1.02469508e-01+0.j]]),
 array([[ 0.01581139+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j, -0.01581139+0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j, -0.01581139+0.j,
          0.        +0.j],
        [ 0.        +0.j,  

In [174]:
file = 'cv_kraus_zz'
np.save(file, kraus, allow_pickle=True, fix_imports=True)